In [1]:
import numpy as np
import pandas as pd

import torch

from transformers import AutoImageProcessor
from datasets import Dataset, DatasetDict, Features, Image, Sequence, Value, concatenate_datasets
import albumentations as A
from albumentations.pytorch import ToTensorV2
import PIL
import cv2

import os

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
data_path = "dataset_original/"

In [3]:
train_df = pd.read_csv(data_path + "train/_classes.csv")
valid_df = pd.read_csv(data_path + "valid/_classes.csv")
test_df = pd.read_csv(data_path + "test/_classes.csv")

In [4]:
train_df

,filename,Crack,Red-Dots,Toothmark
0,CC_1203_jpg.rf.61053fa87a643e3a9bafe7699832936...,1,1,1
1,CC_1124_jpg.rf.60ea6bd496edff0b1a1893d391fb829...,0,0,1
2,CC_1180_jpg.rf.60bf78bedca0f5a672a15037111c54c...,0,0,0
3,CC_1695_jpg.rf.60f3e95ea1ac1d56c7740429592cf4f...,0,1,1
4,CC_2205_jpg.rf.60d5c2db9eecdd981f902b24b8e66c0...,1,0,1
...,...,...,...,...
5217,CC_2482_jpg.rf.c53be5cf801b42b87c902fdb1678787...,0,1,1
5218,CC_1357_jpg.rf.c760c2d9ee7c1969f74f6f4d9eb4d30...,0,1,1
5219,CC_846_jpg.rf.c6ad499bee05275675c71e098864c395...,0,1,1
5220,CC_1056_jpg.rf.c82d5d3936fb6a0f651117f6b054508...,0,0,0


In [5]:
label_columns = ['Crack', 'Red-Dots', 'Toothmark']

In [6]:
# Convert binary-encoded features to array of labels
def binary_to_labels(row, label_cols):
    return [float(row[col]) for col in label_cols]

for df in [train_df, valid_df, test_df]:
    df['labels'] = df.apply(lambda row: binary_to_labels(row, label_columns), axis=1)
    df.drop(columns=label_columns, inplace=True)

In [7]:
# Append image path to filename
def add_image_path(df, split):
    df['image'] = data_path + split + "/" + df['filename']
    df = df[df['image'].apply(os.path.exists)]
    return df[['image', 'labels']]

train_dataset_df = add_image_path(train_df, "train")
valid_dataset_df = add_image_path(valid_df, "valid")
test_dataset_df = add_image_path(test_df, "test")

In [8]:
train_dataset_df

,image,labels
0,dataset/train/CC_1203_jpg.rf.61053fa87a643e3a9...,"[1.0, 1.0, 1.0]"
1,dataset/train/CC_1124_jpg.rf.60ea6bd496edff0b1...,"[0.0, 0.0, 1.0]"
2,dataset/train/CC_1180_jpg.rf.60bf78bedca0f5a67...,"[0.0, 0.0, 0.0]"
3,dataset/train/CC_1695_jpg.rf.60f3e95ea1ac1d56c...,"[0.0, 1.0, 1.0]"
4,dataset/train/CC_2205_jpg.rf.60d5c2db9eecdd981...,"[1.0, 0.0, 1.0]"
...,...,...
5217,dataset/train/CC_2482_jpg.rf.c53be5cf801b42b87...,"[0.0, 1.0, 1.0]"
5218,dataset/train/CC_1357_jpg.rf.c760c2d9ee7c1969f...,"[0.0, 1.0, 1.0]"
5219,dataset/train/CC_846_jpg.rf.c6ad499bee05275675...,"[0.0, 1.0, 1.0]"
5220,dataset/train/CC_1056_jpg.rf.c82d5d3936fb6a0f6...,"[0.0, 0.0, 0.0]"


In [9]:
num_classes = 3
class_names = ['Crack', 'Red-Dots', 'Toothmark']

# Labels is an array of floats
features = Features({
    'image': Image(),
    'labels': Sequence(feature=Value('float32'), length=num_classes)
})

In [10]:
train_dataset = Dataset.from_pandas(train_dataset_df, features=features, preserve_index=False)
valid_dataset = Dataset.from_pandas(valid_dataset_df, features=features, preserve_index=False)
test_dataset = Dataset.from_pandas(test_dataset_df, features=features, preserve_index=False)

In [11]:
def rgb_to_lab(example):
    image_np = np.array(example['image'])
    
    # Remove alpha channel if present
    if image_np.shape[-1] == 4:
        image_np = image_np[..., :3]
    
    image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
    image_lab = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2LAB).astype(np.float32)
    
    # Adjust the LAB channels to standard ranges
    # L channel [0, 100]
    image_lab[..., 0] = image_lab[..., 0] * (100 / 255)
    # a and b channels [-128, 127]
    image_lab[..., 1:] = image_lab[..., 1:] - 128
    
    example['image'] = image_lab
    return example

In [12]:
train_dataset_lab = train_dataset.map(rgb_to_lab)

Map:   0%|          | 0/5222 [00:00<?, ? examples/s]